# Creative Articulator 

## Overview

In this demo, we will look at all the processes in Creative Articulator namely:
 - the creation of a parallel corpus
 - the division of text into fragments 
 - translation of text.

## Parallel corpus

A parallel corpus is a handy tool that allows you to link parts from different corpus. With it, all the corpus will be stored in a single zip file.

In this demonstration, we will work with texts and retellings of Fyodor Mikhailovich Dostoevsky's novel Crime and Punishment, which are located in the `sourse` folder.


The first step is to create a corpus of texts and retellings that are stored in md format.

#### Сorpus for English text:

In [2]:
from tg.grammar_ru.corpus import CorpusBuilder
from pathlib import Path


CorpusBuilder.convert_interformat_folder_to_corpus(
    Path('./files/eng_crime_and_puhishment.base.zip'),
    Path('./source/book/eng'),
    ['book']
)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
from tg.grammar_ru.corpus import CorpusReader


eng_book_reader = CorpusReader(Path('./files/eng_crime_and_puhishment.base.zip'))
eng_book = eng_book_reader.get_toc().index

eng_book_reader.get_toc()[:5]

,filename,timestamp,part_index,token_count,character_count,ordinal,header_0,header_1,headers,book,max_id
file_id,,,,,,,,,,,
1f8d3b1d-7b50-4f62-bef7-70bfc8b62851,eng_crime_and_punishment.md,2023-12-27 04:38:44.526937,0,4028,15089,0,Part One,I,Part One / I,eng_crime_and_punishment,4029
5e58d6b0-f3a7-48ef-89a1-0e7e35ec1ccb,eng_crime_and_punishment.md,2023-12-27 04:38:44.573626,1,8965,32957,1,Part One,II,Part One / II,eng_crime_and_punishment,22995
002184df-ae43-4dcb-8480-50242a27de4f,eng_crime_and_punishment.md,2023-12-27 04:38:44.616217,2,6439,24251,2,Part One,III,Part One / III,eng_crime_and_punishment,39435
b6380e96-cd6b-4852-85af-a58e540b61a6,eng_crime_and_punishment.md,2023-12-27 04:38:44.650223,3,6473,23395,3,Part One,IV,Part One / IV,eng_crime_and_punishment,55909
db7129fb-f00d-49bb-8c6d-a2c65552ba9e,eng_crime_and_punishment.md,2023-12-27 04:38:44.683651,4,5342,19413,4,Part One,V,Part One / V,eng_crime_and_punishment,71252


#### Corpus for English retelling:

In [4]:
CorpusBuilder.convert_interformat_folder_to_corpus(
    Path('./files/eng_retell.base.zip'),
    Path('./source/retell/eng'),
    ['book']
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
eng_retell_reader = CorpusReader(Path('./files/eng_retell.base.zip'))
eng_retell = eng_retell_reader.get_toc().index
eng_retell_reader.get_toc()[:5]

,filename,timestamp,part_index,token_count,character_count,ordinal,header_0,header_1,headers,book,max_id
file_id,,,,,,,,,,,
88359132-e2d0-45f8-937b-45e6a7c31811,eng_retell.md,2023-12-27 04:38:48.807790,0,776,3295,0,Part I,Chapter 1,Part I / Chapter 1,eng_retell,777
caca3fa3-63f6-4cb9-819c-a29dbb7701c0,eng_retell.md,2023-12-27 04:38:48.813467,1,1191,5178,1,Part I,Chapter 2,Part I / Chapter 2,eng_retell,11969
0011b82e-58b1-41b0-b028-da223972c7c3,eng_retell.md,2023-12-27 04:38:48.834699,2,1042,4464,2,Part I,Chapter 3,Part I / Chapter 3,eng_retell,23012
cc5afde8-205c-48b2-be17-fd99c09deeb7,eng_retell.md,2023-12-27 04:38:48.854160,3,1098,4711,3,Part I,Chapter 4,Part I / Chapter 4,eng_retell,34111
ee69b651-fa2b-4b42-9fbe-4869f4d22024,eng_retell.md,2023-12-27 04:38:48.873442,4,954,3932,4,Part I,Chapter 5,Part I / Chapter 5,eng_retell,45066


#### Сorpus for Russian text:

In [6]:
CorpusBuilder.convert_interformat_folder_to_corpus(
    Path('./files/ru_crime_and_puhishment.base.zip'),
    Path('./source/book/ru'),
    ['book']
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
ru_book_reader = CorpusReader(Path('./files/ru_crime_and_puhishment.base.zip'))
ru_book = ru_book_reader.get_toc().index
ru_book_reader.get_toc()[:5]

,filename,timestamp,part_index,token_count,character_count,ordinal,header_0,header_1,headers,book,max_id
file_id,,,,,,,,,,,
1ca1cfb1-b910-476c-8d12-faa09e628f6c,ru_crime_and_punishment.md,2023-12-27 04:39:01.424076,0,3243,13794,0,ЧАСТЬ ПЕРВАЯ,I,ЧАСТЬ ПЕРВАЯ / I,ru_crime_and_punishment,3244
60495d2f-9f3b-47b2-b2a2-2edd5b9c49e9,ru_crime_and_punishment.md,2023-12-27 04:39:01.436402,1,7122,29360,1,ЧАСТЬ ПЕРВАЯ,II,ЧАСТЬ ПЕРВАЯ / II,ru_crime_and_punishment,20367
5ae0a008-b503-49af-b87f-f808ed536a6f,ru_crime_and_punishment.md,2023-12-27 04:39:01.478894,2,5526,22721,2,ЧАСТЬ ПЕРВАЯ,III,ЧАСТЬ ПЕРВАЯ / III,ru_crime_and_punishment,35894
8735bd01-9031-48fe-aaf2-cd0d5dfbdddc,ru_crime_and_punishment.md,2023-12-27 04:39:01.520200,3,5231,21071,3,ЧАСТЬ ПЕРВАЯ,IV,ЧАСТЬ ПЕРВАЯ / IV,ru_crime_and_punishment,51126
44673954-5172-4ebe-9459-f11989cf7a3a,ru_crime_and_punishment.md,2023-12-27 04:39:01.556923,4,4285,17322,4,ЧАСТЬ ПЕРВАЯ,V,ЧАСТЬ ПЕРВАЯ / V,ru_crime_and_punishment,65412


#### Corpus for Russian retelling:

In [8]:
CorpusBuilder.convert_interformat_folder_to_corpus(
    Path('./files/ru_retell.base.zip'),
    Path('./source/retell/ru'),
    ['book']
    )

  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
ru_retell_reader = CorpusReader(Path('./files/ru_retell.base.zip'))
ru_retell = ru_retell_reader.get_toc().index
ru_retell_reader.get_toc()[:5]

,filename,timestamp,part_index,token_count,character_count,ordinal,header_0,header_1,headers,book,max_id
file_id,,,,,,,,,,,
6c4b1afe-0bb3-430e-b586-2c53726530dc,ru_retell.md,2023-12-27 04:39:04.008935,0,396,1814,0,Часть 1,Глава 1: Знакомство с Родионом,Часть 1 / Глава 1: Знакомство с Родионом,ru_retell,397
fc85ff7e-5a48-4f54-82b1-a7911a24d5e4,ru_retell.md,2023-12-27 04:39:04.016608,1,704,3090,1,Часть 1,Глава 2: Встреча с Мармеладовым,Часть 1 / Глава 2: Встреча с Мармеладовым,ru_retell,11102
857c7d93-7fae-4023-b5b4-6a437ea788ce,ru_retell.md,2023-12-27 04:39:04.040925,2,280,1330,2,Часть 1,Глава 3: Письмо матери,Часть 1 / Глава 3: Письмо матери,ru_retell,21383
51db2870-8ad0-41a6-9c1f-2d7a773f6841,ru_retell.md,2023-12-27 04:39:04.061034,3,461,2020,3,Часть 1,Глава 4: Мысли о сестре,Часть 1 / Глава 4: Мысли о сестре,ru_retell,31845
2a6fe819-3c71-4f69-b553-8ef364d3ea58,ru_retell.md,2023-12-27 04:39:04.082069,4,193,892,4,Часть 1,Глава 5: Сон Раскольникова,Часть 1 / Глава 5: Сон Раскольникова,ru_retell,42039


Next, we form a parallel corpus

In [12]:
import pandas as pd

def add_relation(df_1,df_2,name_1,name_2):
    rel_1 = pd.DataFrame({'file_1':df_1, 'file_2':df_2,'relation_name':f"{name_1}_{name_2}"})
    rel_2 = pd.DataFrame({'file_1':df_2, 'file_2':df_1,'relation_name':f"{name_2}_{name_1}"})
    rel = pd.concat([rel_1,rel_2])
    return rel

def add_dfs(name):
    frames = list(name.get_frames())
    dfs = dict(zip(name.get_toc().index,frames))

    return dfs

In [13]:
CorpusBuilder.update_parallel_data(
    Path('./files/parallel_corpus.zip'),
    add_dfs(ru_book_reader),
    "ru_book",
    None)

CorpusBuilder.update_parallel_data(
    Path('./files/parallel_corpus.zip'),
    add_dfs(ru_retell_reader),
    "ru_retell",
    add_relation(ru_book_reader.get_toc().index,ru_retell_reader.get_toc().index,"ru_book","ru_retell"))

CorpusBuilder.update_parallel_data(
    Path('./files/parallel_corpus.zip'),
    add_dfs(eng_book_reader),
    "eng_book",
    add_relation(ru_book_reader.get_toc().index,eng_book_reader.get_toc().index,"ru_book","eng_book"))


CorpusBuilder.update_parallel_data(
    Path('./files/parallel_corpus.zip'),
    add_dfs(eng_retell_reader),
    "eng_retell",
    add_relation(eng_book_reader.get_toc().index,eng_retell_reader.get_toc().index,"eng_book","eng_retell"))



c:\Users\nikya\anaconda3\envs\grammar\lib\zipfile.py:1514: UserWarning: Duplicate name: 'src/1ca1cfb1-b910-476c-8d12-faa09e628f6c.parquet'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\nikya\anaconda3\envs\grammar\lib\zipfile.py:1514: UserWarning: Duplicate name: 'src/60495d2f-9f3b-47b2-b2a2-2edd5b9c49e9.parquet'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\nikya\anaconda3\envs\grammar\lib\zipfile.py:1514: UserWarning: Duplicate name: 'src/5ae0a008-b503-49af-b87f-f808ed536a6f.parquet'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\nikya\anaconda3\envs\grammar\lib\zipfile.py:1514: UserWarning: Duplicate name: 'src/8735bd01-9031-48fe-aaf2-cd0d5dfbdddc.parquet'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\nikya\anaconda3\envs\grammar\lib\zipfile.py:1514: UserWarning: Duplicate name: 'src/44673954-5172-4ebe-9459-f11989cf7a3a.parquet'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


We check that all parts have been successfully recorded

In [14]:
reader = CorpusReader(Path('./files/parallel_corpus.zip'))

df = reader.get_toc()

df.subcorpus_name.unique()

array(['ru_book', 'ru_retell', 'eng_book', 'eng_retell'], dtype=object)

## Translate text

In [15]:
path_parallel_corpus = Path('./files/parallel_corpus.zip')

In [ ]:
#!pip install googletrans==3.1.0a0

In [16]:
from tg.ca.utils_translate import translate_subcorpus

translate_subcorpus(path_parallel_corpus,"eng_retell")

  0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\nikya\anaconda3\envs\grammar\lib\zipfile.py:1514: UserWarning: Duplicate name: 'src/1ca1cfb1-b910-476c-8d12-faa09e628f6c.parquet'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\nikya\anaconda3\envs\grammar\lib\zipfile.py:1514: UserWarning: Duplicate name: 'src/60495d2f-9f3b-47b2-b2a2-2edd5b9c49e9.parquet'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\nikya\anaconda3\envs\grammar\lib\zipfile.py:1514: UserWarning: Duplicate name: 'src/5ae0a008-b503-49af-b87f-f808ed536a6f.parquet'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\nikya\anaconda3\envs\grammar\lib\zipfile.py:1514: UserWarning: Duplicate name: 'src/8735bd01-9031-48fe-aaf2-cd0d5dfbdddc.parquet'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
c:\Users\nikya\anaconda3\envs\grammar\lib\zipfile.py:1514: UserWarning: Duplicate name: 'src/44673954-5172-4ebe-9459-f11989cf7a3a.parquet'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


In [17]:
reader = CorpusReader(Path('./files/parallel_corpus.zip'))

df = reader.get_toc()

df.subcorpus_name.unique()

array(['ru_book', 'ru_retell', 'eng_book', 'eng_retell', 'ru_translate'],
      dtype=object)

### Remove Corpuses

In [ ]:
import os
from pathlib import Path

os.remove(Path('./files/eng_crime_and_puhishment.base.zip'))
os.remove(Path('./files/eng_retell.base.zip'))
os.remove(Path('./files/ru_crime_and_puhishment.base.zip'))
os.remove(Path('./files/ru_retell.base.zip'))
os.remove(Path('./files/translate.base.zip'))